# Run Functions to Add Information to Projects

In [1]:
# ! pip install nltk

In [2]:
import numpy as np
import pandas as pd
from siuba import *

from calitp_data_analysis.sql import to_snakecase

from shared_utils import geography_utils

import dla_utils

import _script_utils
import _data_utils

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/dask_geopandas/backends.py:13: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/l

In [3]:
pd.set_option("display.max_columns", 100)
pd.set_option('display.max_colwidth', None)

## Read in Data and function development

In [4]:
GCS_FILE_PATH  = 'gs://calitp-analytics-data/data-analyses/dla/dla-iija'

In [5]:
### Read in data (First round)
# proj = to_snakecase(pd.read_excel(f"{GCS_FILE_PATH}/FMIS_Projects_Universe_(IIJA_Reporting)_ (3-9-23).xlsx", header=[3], skiprows=range(4,1784)))
# proj.drop(columns =['unnamed:_0'], axis=1, inplace=True)
# proj['summary_recipient_defined_text_field_1_value'] = proj['summary_recipient_defined_text_field_1_value'].fillna(value='None')


In [6]:
proj = to_snakecase(pd.read_excel(f"{GCS_FILE_PATH}/IIJA_FMIS_AllProject_20230309_ToDLA.xlsx",))


In [7]:
proj.sample()

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,project_status_description,project_description,improvement_type,improvement_type_description,total_cost_amount,obligations_amount,summary_recipient_defined_text_field_1_value
1059,44803,YS30,HIGHWAY SAFETY IMP PROG IIJA,P299211,0219000027S,IN SHASTA COUNTY NEAR INGOT FROM 2.7 MILES EAST OF SEAMANS GULCH ROAD TO 1.2 MILES WEST OF DU BOIS ROAD. CURVE IMPROVEMENT,89,Cong Dist 1,Active,ON STATE ROUTE: 299. IN SHASTA COUNTY NEAR INGOT FROM 2.7 MILES EAST OF SEAMANS GULCH ROAD TO 1.2 MILES WEST OF DU BOIS ROAD. CURVE IMPROVEMENT,42,Training,5900.0,5300.0,S SHASTA


In [8]:
# proj.congressional_district.

In [9]:
## move to _data_utils
# def add_new_codes(df):
#     new_codes = to_snakecase(pd.read_excel(f"{GCS_FILE_PATH}/FY21-22ProgramCodesAsOf5-25-2022.v2.xlsx"))
#     code_map = dict(new_codes[['iija_program_code', 'new_description']].values)
    
#     df['program_code_description'] = df.program_code.map(code_map)
#     proj['summary_recipient_defined_text_field_1_value'] = proj['summary_recipient_defined_text_field_1_value'].astype(str)
    
#     return df


In [10]:
proj2 = _data_utils.add_new_codes(proj)

In [11]:

# df = utils.read_data_all()
    
    ## function that adds known agency name to df 
df = _script_utils.identify_agency(proj2, 'summary_recipient_defined_text_field_1_value')

In [12]:
df.sample(3)

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,project_status_description,project_description,improvement_type,improvement_type_description,total_cost_amount,obligations_amount,summary_recipient_defined_text_field_1_value,implementing_agency_locode,implementing_agency,district,county_name,rtpa_name,mpo_name
416,44756,YS30,Highway Safety Improvement Program (HSIP),5276033,0721000114L,"CITY OF GARDENA, VERMONT AVENUE JUST NORTH OF 133RD STREET. INSTALL A PEDESTRIAN CROSSWALK WITH A PEDESTRIAN HYBRID BEACON (PHB) OR HIGH-INTENSITY AC",37,Cong Dist 43,Active,"CITY OF GARDENA, VERMONT AVENUE JUST NORTH OF 133RD STREET. INSTALL A PEDESTRIAN CROSSWALK WITH A PEDESTRIAN HYBRID BEACON (PHB) OR HIGH-INTENSITY ACTIVATED CROSSWALK BEACON (HAWK) CROSSING SYSTEM.",17,Construction Engineering,30000.0,5000.0,L5276SCAG,5276.0,Gardena,7.0,Los Angeles County,Los Angeles County Metropolitan Transportation Auth.,Southern California Association Of Governments
510,44691,Y001,National Highway Performance Program (NHPP),X073148,1112000155S,"IN SAN DIEGO COUNTY AT VARIOUS LOCATIONS. CONSTRUCT MAINTENANCE VEHICLE PULLOUTS(MVPS), PAVE SLOPES UNDER BRIDGE STRUCTURES, NARROW AREAS AND BEYOND",73,Cong Dist 52,Active,"ON STATE ROUTE: 52. IN SAN DIEGO COUNTY AT VARIOUS LOCATIONS. CONSTRUCT MAINTENANCE VEHICLE PULLOUTS(MVPS), PAVE SLOPES UNDER BRIDGE STRUCTURES, NARROW AREAS AND BEYOND GORES. PROJECT LOCATIONS AT SR-52 PM 0.4/5.5 AND I-805 PM 23.5/24.4",42,Training,900.0,700.0,S SANDAG,NaN,California,11.0,San Diego County,NaN,NaN
537,44790,Y400,Congestion Mitigation & Air Quality Improvement (CMAQ),5301024,0621000089L,"FOUR ALLEYS NORTH OF PARK BLVD. BETWEEN 8TH ST. AND CENTER ST., BETWEEN 6TH ST. AND 5TH ST., BETWEEN 5TH ST. AND 4TH ST., AND BETWEEN 4TH ST. AND 3RD",19,Cong Dist 22,Active,"FOUR ALLEYS NORTH OF PARK BLVD. BETWEEN 8TH ST. AND CENTER ST., BETWEEN 6TH ST. AND 5TH ST., BETWEEN 5TH ST. AND 4TH ST., AND BETWEEN 4TH ST. AND 3RD ST. PAVE UNPAVED DIRT ALLEY WAYS",17,Construction Engineering,108690.0,96223.2,L5301COFCG,5301.0,Orange Cove,6.0,Fresno County,Council of Fresno County Governments,Council Of Fresno County Goverments


##### Check unknowns

In [13]:
len(df>>filter(_.implementing_agency=='Unknown'))

1

In [14]:
len(df>>filter(_.county_name=='Unknown'))

1

In [15]:
df>>filter(_.implementing_agency=="Unknown")

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,project_status_description,project_description,improvement_type,improvement_type_description,total_cost_amount,obligations_amount,summary_recipient_defined_text_field_1_value,implementing_agency_locode,implementing_agency,district,county_name,rtpa_name,mpo_name
706,44827,YR10,State RTP Administration,2023000,NaN,2023 Rec Trails Project: State Parks Incurred Administrative Expenses,999,Cong Dist 0,Active,"2023 Rec Trails Project: State Parks Incurred Administrative Expenses. State costs incurred administering the RTP such as staff time, meeting and travel costs and attendance at trail-related training sessions and conferences, Trail Advisory committee costs including meeting and travel costs for committee members, trail conference support, etc.",44,Other,274727.3,241760.0,nan,NaN,Unknown,NaN,Unknown,NaN,NaN


In [16]:
df>>filter(_.project_title.str.contains("Administrative Expense"))

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,project_status_description,project_description,improvement_type,improvement_type_description,total_cost_amount,obligations_amount,summary_recipient_defined_text_field_1_value,implementing_agency_locode,implementing_agency,district,county_name,rtpa_name,mpo_name
706,44827,YR10,State RTP Administration,2023000,NaN,2023 Rec Trails Project: State Parks Incurred Administrative Expenses,999,Cong Dist 0,Active,"2023 Rec Trails Project: State Parks Incurred Administrative Expenses. State costs incurred administering the RTP such as staff time, meeting and travel costs and attendance at trail-related training sessions and conferences, Trail Advisory committee costs including meeting and travel costs for committee members, trail conference support, etc.",44,Other,274727.3,241760.0,nan,NaN,Unknown,NaN,Unknown,NaN,NaN


#### Reformat Columns

To change: 
* ~~District: 2 digit / integer~~
* ~~Congressional District number~~
* Add Progam Code Description and amount to a new project description

In [17]:
df = _data_utils.change_col_to_integer(df, "congressional_district")

In [18]:
df.sample()

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,project_status_description,project_description,improvement_type,improvement_type_description,total_cost_amount,obligations_amount,summary_recipient_defined_text_field_1_value,implementing_agency_locode,implementing_agency,district,county_name,rtpa_name,mpo_name
146,44627,Y001,National Highway Performance Program (NHPP),5906119,0214000079L,"GAS POINT ROAD BRIDGE (06C-0183) OVER NO NAME DITCH, 1.9 MILES WEST OF I-5. BR.# 06C0183 BRIDGE REPLACEMENT",89,1,Active,"GAS POINT ROAD BRIDGE (06C-0183) OVER NO NAME DITCH, 1.9 MILES WEST OF I-5. BR.# 06C0183 BRIDGE REPLACEMENT",17,Construction Engineering,170756.29,105984.0,L5906SHASTA,5906.0,Shasta County,2.0,Shasta County,Shasta County Regional Transportation Planning Agency,Shasta County Regional Transportation Planning Age


In [19]:
## move to _data_utils
# def change_col_to_integer(df, col):
    
#     df[col] = df[col].str.split(' ').str[-1]
    
#     return df

In [20]:

# (df>>select(_.congressional_district, _.congressional_district_2)).sample(20)

In [21]:
df.sample()

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,project_status_description,project_description,improvement_type,improvement_type_description,total_cost_amount,obligations_amount,summary_recipient_defined_text_field_1_value,implementing_agency_locode,implementing_agency,district,county_name,rtpa_name,mpo_name
139,44627,ER01,NaN,15A7001,0521000177L,RIVER ROAD AT FAIRVIEW PERMANENT RESTORATION,53,20,Active,RIVER ROAD AT FAIRVIEW PERMANENT RESTORATION,4,4R - No Added Capacity,1792800.0,1587165.0,L5944AMBAG,5944.0,Monterey County,5.0,Monterey County,Transportation Agency For Monterey County,Association of Monterey Bay Area Governments


In [22]:
## move to utils,
# def add_new_description_col(df):
#     df["obligations_amount_string"] = df["obligations_amount"].astype(str)
    
#     df["new_description_col"] = df["program_code_description"] + " for $" + df["obligations_amount_string"]
    
#     df.drop(columns =['obligations_amount_string'], axis=1, inplace=True)
    
#     return df

In [23]:
# (add_new_description_col(df))>>filter(_.program_code_description.notnull())

In [24]:
df.sample()

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,project_status_description,project_description,improvement_type,improvement_type_description,total_cost_amount,obligations_amount,summary_recipient_defined_text_field_1_value,implementing_agency_locode,implementing_agency,district,county_name,rtpa_name,mpo_name
467,44690,Y001,National Highway Performance Program (NHPP),4052999,0713000238S,"I-405 AT CRENSHAW BLVD. RAMP IMPROVEMENT; IN LOS ANGELES COUNTY, IN THE CITY OF TORRANCE BETWEEN WESTERN AVENUE AND WEST 182ND STREET / CRENSHAW BLVD.",37,43,Active,"ON STATE ROUTE: 405. I-405 AT CRENSHAW BLVD. RAMP IMPROVEMENT; IN LOS ANGELES COUNTY, IN THE CITY OF TORRANCE BETWEEN WESTERN AVENUE AND WEST 182ND STREET / CRENSHAW BLVD. ADD A NEW I-405 ON-RAMP AT CRENSHAW BLVD. INCREASE STORAGE CAPACITY ON THE EXISTING ON/OFF RAMPS AND ADD AUXILIARY LANES ON THE I-405 ON BOTH DIRECTION",17,Construction Engineering,12000000.0,10988400.0,S SCAG,NaN,California,7.0,Los Angeles County,NaN,NaN


In [25]:
df.program_code_description.value_counts()

National Highway Performance Program (NHPP)                           505
Surface Transportation Block Grant (STBG) Program                     386
Highway Safety Improvement Program (HSIP)                             377
Congestion Mitigation & Air Quality Improvement (CMAQ)                104
Transportation Alternatives (Section 133(h))                           48
Projects to Reduce PM 2.5 Emissions                                    42
Section 164 Penalties - Use for HSIP Activities                        37
Bridge Formula Program                                                 33
Metropolitan Planning Program                                          19
Safe and Accessible Transportation Options - Metropolitan Planning     18
Railway-Highway Crossings Program (RHCP)                                7
National Highway Freight Program (NHFP)                                 6
Vulnerable Road User Safety Special Rule                                5
State Planning and Research (SPR)     

## Test & Export

In [26]:
# GCS_FILE_PATH  = 'gs://calitp-analytics-data/data-analyses/dla/dla-iija'

In [27]:
###test agg. PASS df with title column, concat and 552 rows
##throwingerror
agg = _script_utils.get_clean_data(df, full_or_agg = 'agg')

In [28]:
agg.sample(3)

,fmis_transaction_date,project_number,implementing_agency,summary_recipient_defined_text_field_1_value,program_code,program_code_description,recipient_project_number,improvement_type,improvement_type_description,project_title,obligations_amount,congressional_district,district,county_code,county_name,county_name_title,implementing_agency_locode,rtpa_name,mpo_name,project_title_new,new_description_col
865,44907,5060404,Fresno,L5060FCOG,Y400,Congestion Mitigation & Air Quality Improvement (CMAQ),0623000063L,15,Preliminary Engineering,"CHESTNUT AVENUE NEIGHBORHOOD: AREA BOUNDED BY CHESTNUT AVENUE, TULARE STREET, WILLOW AVENUE, AND BELMONT AVENUE SIDEWALK GAP INFILL AND CURB RAMP REP",318900.0,16,6.0,19,Fresno County,Fresno County,5060.0,Council of Fresno County Governments,Council Of Fresno County Goverments,Sidewalk in Fresno,"This project is part of the Congestion Mitigation & Air Quality Improvement (CMAQ) Program, and recieved $ 318900.0. This project will Sidewalk in Fresno"
152,44666,5925190,El Dorado County,L5925SACOG,Y230,Surface Transportation Block Grant (STBG) Program,0322000170L,4|17,4R - No Added Capacity|Construction Engineering,"AT MISSOURI FLAT RD 0.8 MILES SOUTH OF GOLDEN CENTER DRIVE CONSTRUCT BIKE TRAIL CONNECTION, EXTEND CURB, GUTTER, AND SIDEWALK. TC",356000.0,4,3.0,17,El Dorado County,El Dorado County,5925.0,El Dorado County Transportation Commission,Sacramento Area Council Of Governments,Construct Sidewalk in El Dorado County,"This project is part of the Surface Transportation Block Grant (STBG) Program Program, and recieved $ 356000.0. This project will Construct Sidewalk in El Dorado County"
287,44718,5942198,Fresno County,L5942FCOG,Y001,National Highway Performance Program (NHPP),0600020576L,15,Preliminary Engineering,BRIDGE 42C0175 ON MANNING AVENUE OVER TRAVERS CREEK REPLACE 2 LANE BRIDGE WITH 4 LANE BRIDGE,398385.0,22,6.0,19,Fresno County,Fresno County,5942.0,Council of Fresno County Governments,Council Of Fresno County Goverments,Preliminary Engineering Projects in Fresno County,"This project is part of the National Highway Performance Program (NHPP) Program, and recieved $ 398385.0. This project will Preliminary Engineering Projects in Fresno County"


In [29]:
(agg>>filter(_.county_name_title=="Statewide")).sample()

,fmis_transaction_date,project_number,implementing_agency,summary_recipient_defined_text_field_1_value,program_code,program_code_description,recipient_project_number,improvement_type,improvement_type_description,project_title,obligations_amount,congressional_district,district,county_code,county_name,county_name_title,implementing_agency_locode,rtpa_name,mpo_name,project_title_new,new_description_col
691,44826,0010209,California,S SANDAG,Y450,Metropolitan Planning Program,0022000354S,18,Planning,OWP 22/23 SAN DIEGO REGION CONSOLIDATED PLANNING GRANT (CPG) PROJECT FOR SAN DIEGO ASSOCIATION OF GOVERNMENTS (SANDAG) OVERALL WORK PROGRAM (OWP) FOR,4851322.0,0,nan,999,Statewide,Statewide,nan,None,None,Planning and Research Statewide,"This project is part of the Metropolitan Planning Program Program, and recieved $ 4851322.0. This project will Planning and Research Statewide"


## Program Code deep dive

In [30]:
(agg>>filter(_.program_code_description==("nan"))).program_code.value_counts()

ER01    287
Y928      2
Y520      1
Name: program_code, dtype: int64

In [31]:
# (agg>>filter(_.program_code_description==("nan")))

### Read in Old Codes


(currently used in script)

In [32]:
#check project codes list
codes = to_snakecase(pd.read_excel(f"{GCS_FILE_PATH}/FY21-22ProgramCodesAsOf5-25-2022.v2.xlsx"))


In [33]:
codes.head()

,description,iija_program_code,new_description
0,Advanced Transportation Technologies and Innovative Mobility Deployment,Y44A,Advanced Transportation Technologies and Innovative Mobility Deployment
1,Applalachian Developmen Highway System,Y140,Applalachian Developmen Highway System
2,Border State Infrastructure,Y500,Border State Infrastructure
3,Bridge Formula Program,Y110,Bridge Formula Program
4,Bridge Formula Program Off-System Bridges,Y120,Bridge Formula Program


### New Codes

In [34]:
## read in new project codes:
new_codes = to_snakecase(pd.read_excel(f"{GCS_FILE_PATH}/lst_IIJA_Code_20230501.xlsx"))

In [35]:
new_codes

,iija_code,description,program_name,iija_program_code
0,|BFP|,Bridge Formula Program,Bridge Formula Program,Y120
1,|CPF|,Community Project Funding Congressionally Directed Spending,Community Project Funding Congressionally Directed Spending,Y928
2,|NHPP|,National Highway Performance Program (NHPP),National Highway Performance Program (NHPP),Y001
3,|NHPPE|,NHPP Exempt,NHPP Exempt,Y002
4,NaN,Protective Features [NEW],Protective Features [NEW],Y008
...,...,...,...,...
100,|HRDP|,Highway Research and Deveopment Program,Highway Research and Development Program,Y448
101,NaN,Bridge Replacement and Rehabilitation Program,NaN,Y908
102,NaN,Technology and Innovation Deployment - FMIS,NaN,"Y37G, Y37H, Y379, Y37P"
103,|ER|,Emergency Suppliment Funding,Emergency Supplement Funding,ER01


#### Remove | from codes

In [36]:
# new_codes['iija_code_2'] = new_codes['iija_code'].str.split("|").str[0]
new_codes['iija_code_2'] = new_codes['iija_code'].str.replace("|","")

/tmp/ipykernel_325/474405769.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.


In [37]:
new_codes.sample(5)

,iija_code,description,program_name,iija_program_code,iija_code_2
57,|SATOSPR|,Safe and Accessible Transportation Options - SPR,Safe and Accessible Transportation Options - SPR,Y570,SATOSPR
74,|HRDP|,Highway Research and Development,Highway Research and Development Program,Z48F,HRDP
101,NaN,Bridge Replacement and Rehabilitation Program,NaN,Y908,NaN
21,|IAE|,Improving Accessibility and Efficiency- Urbanized Areas With Population Over 200K [NEW],Improving Accessibility and Efficiency,Y101,IAE
26,|RTP|,Recreational Trails Program (RTP),Recreational Trails Program,Y940,RTP


#### Codes with ranges

In [38]:
new_codes>>filter(_.iija_program_code.str.contains("-"))

,iija_code,description,program_name,iija_program_code,iija_code_2
75,NaN,IIJA Puerto Rico Highway,NaN,YP10 -YP30,NaN
91,NaN,HIP - Flex & Pulation based,NaN,Z928 - Z931,NaN
95,NaN,National Infrastractur Investment (RAISE),NaN,RA01 - RA03,NaN


In [39]:
## checking with the proj
proj>>filter(_.program_code.str.contains("Z"))

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,project_status_description,project_description,improvement_type,improvement_type_description,total_cost_amount,obligations_amount,summary_recipient_defined_text_field_1_value


In [40]:
# proj

#### Null codes

In [41]:
len(new_codes>>filter(_.iija_code.isna()))

48

In [42]:
null_program_codes = new_codes>>filter(_.iija_code.isna())

In [43]:
#download and send to contact! 
# null_program_codes.to_csv("lst_IIJA_Code_20230501_null.csv")

In [44]:
# null_program_codes

### Add acronym code for IIJA programs
https://stackoverflow.com/questions/4355201/creating-acronyms-in-python

In [45]:
# (proj>>filter(_.program_code==("ER01"), _.program_code_description.notnull())).sample()

In [46]:
# (proj>>filter(_.program_code==("ER01"), _.program_code_description.notnull())).sample()

In [47]:
df.head()

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,project_status_description,project_description,improvement_type,improvement_type_description,total_cost_amount,obligations_amount,summary_recipient_defined_text_field_1_value,implementing_agency_locode,implementing_agency,district,county_name,rtpa_name,mpo_name,county_name_title
0,44594,ER01,nan,32L0118,0117000181L,MATTOLE ROAD PM 5.00 PERMANENT RESTORATION,23,2,Active,MATTOLE ROAD PM 5.00 PERMANENT RESTORATION,4,4R - No Added Capacity,68757.00,60870.23,L5904NON-MPO,5904.0,Humboldt County,1.0,Humboldt County,Humboldt County Association of Governments,NON-MPO,Humboldt County
1,44594,ER01,nan,32L0118,0117000181L,MATTOLE ROAD PM 5.00 PERMANENT RESTORATION,23,2,Active,MATTOLE ROAD PM 5.00 PERMANENT RESTORATION,15,Preliminary Engineering,207848.00,184008.19,L5904NON-MPO,5904.0,Humboldt County,1.0,Humboldt County,Humboldt County Association of Governments,NON-MPO,Humboldt County
2,44594,ER01,nan,32L0118,0117000181L,MATTOLE ROAD PM 5.00 PERMANENT RESTORATION,23,2,Active,MATTOLE ROAD PM 5.00 PERMANENT RESTORATION,17,Construction Engineering,139775.00,123742.50,L5904NON-MPO,5904.0,Humboldt County,1.0,Humboldt County,Humboldt County Association of Governments,NON-MPO,Humboldt County
3,44594,ER01,nan,4400046,01924731L,ALDERPOINT ROAD PM 40.21 TO 40.70 EMERGENCY RELIEF PR ONLY,23,2,Active,ALDERPOINT ROAD PM 40.21 TO 40.70 EMERGENCY RELIEF PR ONLY,4,4R - No Added Capacity,124980.90,110645.59,L5904NON-MPO,5904.0,Humboldt County,1.0,Humboldt County,Humboldt County Association of Governments,NON-MPO,Humboldt County
4,44594,ER01,nan,4400046,01924731L,ALDERPOINT ROAD PM 40.21 TO 40.70 EMERGENCY RELIEF PR ONLY,23,2,Active,ALDERPOINT ROAD PM 40.21 TO 40.70 EMERGENCY RELIEF PR ONLY,15,Preliminary Engineering,831.49,736.11,L5904NON-MPO,5904.0,Humboldt County,1.0,Humboldt County,Humboldt County Association of Governments,NON-MPO,Humboldt County


##### Check titles
moved to `_script_utils`

In [48]:
# sorted(list(agg.project_title_new.unique()))

In [49]:
# agg>>filter(_.implementing_agency.str.contains("California"))

In [50]:
# agg>>filter(_.project_title_new.str.contains("Caltrans"))

In [51]:
# len(agg)

In [52]:
# agg = utils.title_column_names(agg)

In [53]:
# agg.to_csv(f"{GCS_FILE_PATH}/IIJA_FMIS_AllProject_20230315_ToDLA_agg.csv")

In [54]:
###test full. PASS title, no concat and 1241 rows
# full = utils.get_clean_data(df, full_or_agg = 'full')

In [55]:
# full = utils.title_column_names(full)

In [56]:
# full.to_csv(f"{GCS_FILE_PATH}/FMIS_Projects_Universe_IIJA_Reporting_3-9-23_all.csv")

In [57]:
# agg = to_snakecase(agg)

In [58]:
# test_df = agg>>filter(_.implementing_agency.str.contains('Unknown')
                      # | _.implementing_agency.str.contains("Caltrans")
                     # )

In [59]:
# test_df.sample()

In [60]:
# locodes = to_snakecase(pd.read_excel(f"gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/locodes_updated7122021.xlsx"))

In [61]:
# locodes['agency_name'] = locodes['agency_name'].str.upper()

In [62]:
# locode_names = sorted(list(locodes['agency_name'].unique()), reverse=True)

In [63]:
# locode_names.remove('ROSS')

In [64]:
# locode_names.append("State Parks")

In [65]:
# locode_names

In [66]:
# pattern = '|'.join(locode_names)

In [67]:
# test_df.project_title.str.contains(pattern)

In [68]:
### https://stackoverflow.com/questions/68869434/create-an-pandas-column-if-a-string-from-a-list-matches-from-another-column
#test_df["agencies_found"] = test_df["project_title"].apply(lambda x: next((a for a in locode_names if a in x), ""))
# test_df["agencies_found2"] = test_df["project_title"].map(lambda s: next((name for name in locode_names if name in s), np.nan))

In [69]:
# test_df["agencies_found2"] = test_df["agencies_found2"].str.title()

In [70]:
# test_df

In [71]:
# agg>>filter(_["Project Title New"].str.contains('Unknown'))

In [72]:
# agg>>filter(_["implementing_agency"].str.contains('Unknown'))

### Next